In [1]:
!pip install Pillow

In [2]:
!pip install pytesseract transformers sentence-transformers faiss-cpu chromadb openai langchain

In [3]:
!pip install langchain openai faiss-cpu tiktoken python-dotenv pypdf langchain-community langchain_openai
!pip install langchain-google-genai
!pip install groq langchain-groq

In [15]:
#!pip install sentence-transformers

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import numpy as np
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import pandas as pd
from google.colab import userdata
import os
from langchain_groq import ChatGroq
import re

In [5]:
groq_api_key = userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] = groq_api_key

In [6]:
import pytesseract
from PIL import Image

def perform_ocr(image_path):
    """
    Performs OCR on an image file and returns the extracted text.

    Args:
        image_path (str): The path to the image file.

    Returns:
        str: The extracted text from the image.
    """
    try:
        # Open the image file
        img = Image.open(image_path)
        # Perform OCR using pytesseract
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        print(f"Error performing OCR on {image_path}: {e}")
        return ""

In [7]:
import os

# Define the directory containing the bank screenshots
bankimages_dir = 'bankimages' # Assuming the folder is in the same directory as the notebook

extracted_texts_with_paths = []
original_text_snippets = [] # Store only the text for easy retrieval

# Check if the bankimages directory exists
if not os.path.exists(bankimages_dir):
    print(f"Error: The directory '{bankimages_dir}' was not found.")
else:
    # List all files in the directory and filter for image files (e.g., .jpg, .png)
    image_files = [f for f in os.listdir(bankimages_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]

    if not image_files:
        print(f"No image files found in the directory '{bankimages_dir}'.")
    else:
        for image_file in image_files:
            image_path = os.path.join(bankimages_dir, image_file)
            extracted_text = perform_ocr(image_path) # Use the perform_ocr function
            extracted_texts_with_paths.append({'path': image_path, 'text': extracted_text})
            original_text_snippets.append(extracted_text)

        print(f"Processed {len(extracted_texts_with_paths)} images and extracted text.")
        #for item in extracted_texts_with_paths:
        #    print(f"Path: {item['path']}, Text: {item['text'][:50]}...") # Print first 50 chars

Processed 9 images and extracted text.


In [8]:
#Apply Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=100)
documents = [Document(page_content=item['text']) for item in extracted_texts_with_paths] # Correct: creates a Document for each text snippet
texts = text_splitter.split_documents(documents)

 #4. create Vector DB
# Attempt to install compatible versions of sentence-transformers and transformers
!pip install -U sentence-transformers transformers

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert embeddings to numpy array of float32 before creating FAISS index
import numpy as np
text_embeddings_np = np.array([embeddings.embed_query(doc.page_content) for doc in texts]).astype('float32')

db_img_text = FAISS.from_documents(texts, embeddings)


llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")
retriever = db_img_text.as_retriever(search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

/tmp/ipython-input-3177372584.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
def Get_LLM_Response(question):
  response = qa_chain({"query": question})
  return response['result']

In [ ]:
#question = "how to see Debit Card information"
#response = qa_chain({"query": question})
#print(response['result'])

In [10]:
!pip install gradio

In [11]:
import gradio as gr

def process_input(text_input):
  response = qa_chain({"query": text_input})
  result = response['result']
  return result

# Define a clear function to clear both input and output textboxes
def clear_inputs():
    return "", ""

'''
with gr.Blocks() as iface:
    gr.Markdown("<h1 style='text-align: center; background-color: #f0f0f0;'>FAQs</h1>") # Centered heading with background color
    with gr.Row():
        with gr.Column(): # Column for input
            text_input = gr.Textbox(lines=2, placeholder="Enter your text here", label="Input Text")
            submit_button = gr.Button("Submit")
            clear_button = gr.Button("Clear")
        with gr.Column(): # Column for output
            output = gr.Textbox(label="Output")
'''
with gr.Blocks() as iface:
    with gr.Row():
       gr.Markdown("<h1 style='text-align: center; background-color: #f0f0f0;'>FAQs</h1>") # Centered heading with background color
    with gr.Row():
        text_input = gr.Textbox(lines=2, placeholder="Enter your FAQ here", label="FAQ Text")
        output = gr.Textbox(label="Steps")
    with gr.Row():
        submit_button = gr.Button("Submit")
        clear_button = gr.Button("Clear")

    submit_button.click(fn=process_input, inputs=[text_input], outputs=output)
    clear_button.click(fn=clear_inputs, inputs=None, outputs=[text_input, output])


iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://abb24284fe27d5d965.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
